# Coletor de Informações GenBank

###### by Pedro Souza

> <div align="justify"> Coletar informações no GenBank visando criar um repositório um pouco mais robusto para as suas análises moleculares as vezes é complicado devido aos diversos parâmetros existentes para cada deposito e o jeito do GenBank apresentar essas informações. Visando melhorar a visualização inicial desse material pelos pesquisadores desenvolvi esse organizador de informações. Para utilizar basta baixar todo o conteudo que deseja triar utilizando a função compiladora do GenBank e salvar o arquivo completo (.gb ou Genbank), depois basta salvar uma cópia desse material em txt e utilizar a função  <b><font color='green'>GenBank_coletor()</font>.</div>

## Extenções necessárias

In [1]:
import pandas as pd
import re

## Funções necessárias

In [5]:
def data_reader(file): # Função que realizará a leitura do arquivo Genbank
    with open(file) as f:
        GB = f.read()
    
    GB = re.sub('://', '', GB)
    GB_split = GB.split('//')
    
    print(len(GB_split), 'possiveis sequências foram reconhecidas. \n\n' )
    return GB_split


def info_organizer(data): # Função que realizará a organização dos dados retirados do arquivo base
    
    prod = 0
    a = 0
    n = 0
    j = []
    t = []
    seq = []
    tam = []
    time = []
    sp = []
    gene = []
    title = []
    journal = []
    journal_temp = []
    pais = []
    
    for y in data:
        
        if len(y) > 10: # Essa condicional remove qualquer erro na leitura dos registros, já que remove registros com menos de 10 caracteres
            
            lines = re.split('\n+',y) # Para dar menos trabalho da análise dos registros divido os mesmos em linhas
            
            ng = len(re.findall('/product=',y)) # Alguns registros possuem mais de um gene utilizo esse comando para contar
            ps = len(re.findall('/country',y))

            for i in lines: # Com esse loop o programa vai procurar linha por linha as informações requisitadas
                n += 1
                
                if re.search('LOCUS',i): # Essa parte coleta o código do GenBank, o tamanho e a data de depósito
                    info = re.split('\s+',i)
                    seq.append(info[1])
                    tam.append(info[2])
                    time.append(info[7])
                
                
                if re.search('SOURCE',i): # Essa parte coleta o orgânismos análisado
                    info = re.split('\s+',i)
                    sp.append(info[1] + ' '+ info[2])
                    #print(sp)
                
                if re.search('TITLE',i): # Essa parte colete a titulo do trabalho
                    t.append(n)

                if re.search('JOURNAL',i): # Essa parte coleta informação sobre a revista, caso o trabalho já tenha sido aceito
                    journal_temp.append(i)
                    j.append(n)
                
                
            if ng == 1: # Aqui entramos na seleçõo dos genes
                for i in lines:
                    if re.findall('/product=',i):
                        if re.search('small subunit ribosomal RNA',i) or re.search('18S ribosomal RNA',i) or re.search('small subunit ribosomal',i):
                            gene.append('18S')
                        elif re.search('large subunit ribosomal RNA',i):
                            gene.append('28S')
                        elif re.search('cytochrome c oxidase subunit 1',i) or re.search('cytochrome oxidase subunit 1',i):
                            gene.append('COI-1')
                        else:
                            gene_temp = i
                            gene_temp = re.sub('\s+',' ', gene_temp)
                            gene_temp = re.sub('/product=','', gene_temp)
                            gene_temp = re.sub('"','', gene_temp)
                            
                            gene.append(gene_temp)
            elif ng < 1:
                if re.search('/note=',y):
                    gene.append('Possível fragmento')
            else:
                #print(a, ': Multi-gene')
                gene.append('Multi-gene')
                    #gene.append(2)
                    
            if ps == 1:
                for i in lines:
                    if re.search('/country', i):
                        pais_temp = i
                        pais_temp = re.sub('\s+',' ', pais_temp)
                        pais_temp = re.sub('/country=','', pais_temp)
                        pais_temp = re.sub('"','', pais_temp)
                        pais.append(pais_temp)
                        
            if ps < 1:
                pais.append('SR')
            
            if ps > 1:
                pais.append('Mais que um pais')

            #print(a,t)

            a += 1
            prod = 0
           
            # A coleta de informações sobre os títulos e sobre as revistas apresentaram alguns problemas, como repetições ou estar incompleto.
            
            # Resolvendo titulo

            h = int(t[0] - 1)
            g = int(j[0] - 1)
            titulo_temp = ''.join(lines[h:g])
            titulo_temp = re.sub('TITLE', '', titulo_temp)
            titulo_temp = re.sub('\s+', ' ', titulo_temp)
            title.append(titulo_temp)

            n = 0
            t = []
            j = []

            # Resolvendo Revista

            j_temp = re.sub('JOURNAL','',journal_temp[0])
            j_temp = re.sub('\s+',' ',j_temp)

            journal.append(j_temp)

            journal_temp = []

            #
    
    df = pd.DataFrame(list(zip(seq, sp, gene,tam, time,pais,title,journal)),
               columns =['Sequência','Espécie','Gene', 'Tamanho', 'Data','País','Título','Revista'])
    
    print('Informações de {} sequências foram recuperadas.'.format(len(df)))
    print('Destas, {}, são possíveis sequências fragmentadas, {}, apresentam mais de um gene e {}, apresentam algum tipo de erro de identificação do gene.'.format(len(df.query('Gene == "Possível fragmento"')), len(df.query('Gene == "Multi-gene"')), len(df.query('Gene == "Error"'))))
    
    return(df)

def GenBank_coletor(path): # Função final que compila as duas primeiras e salva o resultado em um arquivo excel e retorna, tambem, os dados para análise posterior
    GB = data_reader(path)
    df = info_organizer(GB)
    df.to_excel('GenBank_data_out.xlsx')
    print('\n\n' + 'Seus dados foram organizados no arquivo "GenBank_data_out.xlsx".')
    return df

> Para utilizar o organizador basta passar para a função o arquivo que deseja ser análisado, e.g.:
    <b><p style="color:blue"> GenBank_coletor('dataset_Peritrichia_out2022.txt') </p></b>
    
> A função retornará um dataframe organizado que será salvo na sua pasta e também pode ser adicionada a um objeto. Além disso, retornará diversas informações sobre o sucesso do processo, como número de sequências indentificadas, análisadas e etc.
    

In [9]:
GenBank_coletor('dataset_Peritrichia_out2022.txt')

1344 possiveis sequências foram reconhecidas. 


Informações de 1343 sequências foram recuperadas.
Destas, 134, são possíveis sequências fragmentadas, 354, apresentam mais de um gene e 0, apresentam algum tipo de erro de identificação do gene.


Seus dados foram organizados no arquivo "GenBank_data_out.xlsx".


,Sequência,Espécie,Gene,Tamanho,Data,País,Título,Revista
0,OP289656,Epistylis pygmaeum,18S,1586,14-OCT-2022,China: Jining,Integrative Studies on Three Epibiotic Episty...,"Protist 173 (5), 125909 (2022)"
1,OP289655,Epistylis daphniae,18S,1648,14-OCT-2022,China: Jining,Integrative Studies on Three Epibiotic Episty...,"Protist 173 (5), 125909 (2022)"
2,OP289654,Epistylis weishanensis,18S,1647,14-OCT-2022,China: Jining,Integrative Studies on Three Epibiotic Episty...,"Protist 173 (5), 125909 (2022)"
3,ON158510,Carchesium cyclopidarum,18S,1544,26-SEP-2022,China: Jining,The taxonomy and molecular phylogeny of two e...,"Eur J Protistol 86, 125921 (2022)"
4,ON158509,Epistylis qingdaoensis,18S,1613,26-SEP-2022,China: Qingdao,The taxonomy and molecular phylogeny of two e...,"Eur J Protistol 86, 125921 (2022)"
...,...,...,...,...,...,...,...,...
1338,AF335515,Epistylis wenrichi,18S,1723,16-NOV-2001,SR,Phylogenetic relationships of the subclass pe...,"J. Eukaryot. Microbiol. 48 (5), 583-587 (2001)"
1339,AF335514,Epistylis chrysemydis,18S,1727,16-NOV-2001,SR,Phylogenetic relationships of the subclass pe...,"J. Eukaryot. Microbiol. 48 (5), 583-587 (2001)"
1340,AF335513,Epistylis hentscheli,18S,1729,16-NOV-2001,SR,Phylogenetic relationships of the subclass pe...,"J. Eukaryot. Microbiol. 48 (5), 583-587 (2001)"
1341,AF070701,Vorticella microstoma,18S,1688,01-JUL-1998,SR,Phylogenetic relationships of Vorticella conv...,Unpublished
